# Lab 2 — Errors & Metrics Identification

This notebook implements the assignment from `Lab2_Errors_Metrics_Identification.pdf`. It loads the LFW faces dataset, trains a simple classifier, computes common evaluation metrics (accuracy, precision, recall, F1), shows a confusion matrix, and displays example misclassifications.

In [3]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load dataset (people with >= 70 images to keep problem manageable)
lfw = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
X = lfw.data  # flattened images
y = lfw.target
target_names = lfw.target_names
images = lfw.images
print('Dataset shape:', X.shape)
print('Number of classes:', len(target_names))

In [ ]:
# Train/test split
X_train, X_test, y_train, y_test, img_train, img_test = train_test_split(X, y, images, test_size=0.3, random_state=42, stratify=y)
print('Train samples:', X_train.shape[0])
print('Test samples :', X_test.shape[0])

In [ ]:
# Train a simple classifier (multinomial logistic regression via one-vs-rest)
clf = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='ovr', random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Done training')

In [ ]:
# Compute evaluation metrics
acc = accuracy_score(y_test, y_pred)
prec_macro = precision_score(y_test, y_pred, average='macro')
rec_macro = recall_score(y_test, y_pred, average='macro')
f1_macro = f1_score(y_test, y_pred, average='macro')
print(f'Accuracy: {acc:.4f}')
print(f'Precision (macro): {prec_macro:.4f}')
print(f'Recall (macro): {rec_macro:.4f}')
print(f'F1 (macro): {f1_macro:.4f}')

In [ ]:
# Detailed classification report
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
# Confusion matrix (normalized) and plot
cm = confusion_matrix(y_test, y_pred)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(8,6))
sns.heatmap(cm_norm, annot=True, fmt='.2f', xticklabels=target_names, yticklabels=target_names, cmap='Blues')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Normalized Confusion Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Show some example misclassifications
mis_idx = np.where(y_test != y_pred)[0]
print('Number of misclassifications:', len(mis_idx))
n_show = min(8, len(mis_idx))
if n_show > 0:
    plt.figure(figsize=(12,4))
    for i, idx in enumerate(mis_idx[:n_show]):
        ax = plt.subplot(1, n_show, i+1)
        plt.imshow(img_test[idx], cmap='gray')
        plt.title(f'True: {target_names[y_test[idx]]}
Pred: {target_names[y_pred[idx]]}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print('No misclassifications to display')

In [ ]:
# Analyze most confused pairs (off-diagonal highest counts)
cm_off = cm.copy()
np.fill_diagonal(cm_off, 0)
pairs = []
for i in range(cm_off.shape[0]):
    for j in range(cm_off.shape[1]):
        if cm_off[i,j] > 0:
            pairs.append(((target_names[i], target_names[j]), int(cm_off[i,j])))
pairs = sorted(pairs, key=lambda x: x[1], reverse=True)[:8]
print('Top confused class pairs (True -> Predicted)')
for (t,p),c in pairs:
    print(f'{t} -> {p}: {c}')

**Notes / Observations**
- Accuracy, precision, recall, and F1 are reported above. Use the classification report to inspect per-class behaviour.
- The normalized confusion matrix highlights which classes are commonly mistaken for others.
- Example misclassified images are displayed to help identify why errors occur (e.g., pose, occlusion, lighting).

Feel free to run the notebook cells locally to reproduce results and tune the classifier (e.g., try `RandomForestClassifier`, SVM, or PCA + classifier).